# ARC Tools
### Visuallize OneDMin r_min and r_max
The r (min and max) distance is defined between the center of mass of the species and the center of ass of the bath gas.

In [ ]:
xyz = """
C       0.01110400    0.73099100    0.00000000
H       0.27332400    1.21632500    0.93134200
H       0.27332400    1.21632500   -0.93134200
N       0.01045700   -0.65996900   -0.00000000
H      -0.34564700   -1.09923900    0.83510100
H      -0.34564700   -1.09923900   -0.83510100
"""

bath_gas = 'N2'  # allowed values are 'He', 'Ne', 'Ar', 'Kr', 'H2', 'N2', or 'O2'

# set the values below to bypass ARC's determination and visuallize your own values
r_min = 3
r_max = 6

In [ ]:
from arc.species import ARCSpecies
from arc.common import get_atom_radius, get_center_of_mass
from arc.plotter import draw_3d

def plot_collider(r):
    """
    Plot a species and its collider.
    """
    conf_xyz = spc.conformers[0] if len(spc.conformers) else None
    xyz = spc.initial_xyz or spc.final_xyz or conf_xyz
    cm = get_center_of_mass(xyz)
    
    if '2' in bath_gas:
        symbol = bath_gas[0]
        bath_gas_bond_length = {'H2': 0.743, 'N2': 1.1, 'O2': 1.208}  # Angstrom
        xyz = add_bath_gas_to_xyz(xyz, symbol, cm, r - 0.5 * bath_gas_bond_length[bath_gas])
        xyz = add_bath_gas_to_xyz(xyz, symbol, cm, r + 0.5 * bath_gas_bond_length[bath_gas])
    else:
        xyz = add_bath_gas_to_xyz(xyz, bath_gas, cm, r)
    draw_3d(xyz=xyz, species=spc)

def add_bath_gas_to_xyz(xyz, symbol, cm, r):
    """
    Add an atom (bath gas) at distance r from the center of mass to xyz and return xyz
    """
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0] + r, y=cm[1], z=cm[2])
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0], y=cm[1] + r, z=cm[2])
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0], y=cm[1], z=cm[2] + r)
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0] - r, y=cm[1], z=cm[2])
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0], y=cm[1] - r, z=cm[2])
    xyz += '\n{s} {x:.6f} {y:.6f} {z:.6f}'.format(s=symbol, x=cm[0], y=cm[1], z=cm[2] - r)
    return xyz

spc = ARCSpecies(label='1dmin', xyz=xyz, r_min=r_min, r_max=r_max)
spc.final_xyz = xyz

print('Species: {0}'.format(spc.label))
spc_r = spc.determine_radius()
print('Species radius: {0:.2} A'.format(spc_r))

spc.determine_onedmin_radii(bath_gas)

print('r min: {0}, r max: {1}'.format(spc.r_min, spc.r_max))

print('\n\n r_min: {0}'.format(spc.r_min))
plot_collider(spc.r_min)
print('\n\n r_max: {0}'.format(spc.r_max))
plot_collider(spc.r_max)